In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
train = pd.read_csv("E:\PROJECTS\ML PROJECTS\TITANIC DATASET EDA AND FEATURE ENGINEERING\train (2).csv")
test = pd.read_csv("E:\PROJECTS\ML PROJECTS\TITANIC DATASET EDA AND FEATURE ENGINEERING\test.csv")
train_len = len(train)
df = pd.concat([train, test], sort=False).reset_index(drop=True)
df['Title'] = df['Name'].str.extract(',\s*([^\.]+)\.', expand=False)
title_counts = df['Title'].value_counts()
rare_titles = title_counts[title_counts < 10].index
df['Title'] = df['Title'].replace(list(rare_titles), 'Rare')
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
df['Cabin'] = df['Cabin'].fillna('U')
df['Deck'] = df['Cabin'].astype(str).str[0]
def get_ticket_prefix(t):
    t = str(t).replace('.', '').replace('/', '').strip().upper()
    parts = t.split()
    prefixes = [p for p in parts if not p.isdigit()]
    return prefixes[0] if len(prefixes) > 0 else 'X'
df['TicketPrefix'] = df['Ticket'].apply(get_ticket_prefix)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}).astype(int)
cat_cols = ['Title', 'Embarked', 'Deck', 'TicketPrefix']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
drop_cols = ['Name', 'Ticket', 'Cabin']
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
train_df = df.iloc[:train_len].copy()
test_df = df.iloc[train_len:].copy()
X = train_df.drop(columns=['Survived', 'PassengerId'])
y = train_df['Survived'].astype(int)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
num_cols = ['Age', 'Fare', 'FamilySize']
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
val_preds = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print(classification_report(y_val, val_preds))
joblib.dump(model, "model_rf.joblib")
joblib.dump(scaler, "scaler.joblib")
test_X = test_df.drop(columns=['Survived', 'PassengerId'])
test_X[num_cols] = scaler.transform(test_X[num_cols])
test_preds = model.predict(test_X)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'].astype(int), 'Survived': test_preds.astype(int)})
submission.to_csv("submission.csv", index=False)
print("Submission saved to submission.csv")


<>:9: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Rishu Yadav\AppData\Local\Temp\ipykernel_11044\1141008655.py:9: SyntaxWarning: invalid escape sequence '\P'
  train = pd.read_csv("E:\PROJECTS\ML PROJECTS\TITANIC DATASET EDA AND FEATURE ENGINEERING\train (2).csv")
C:\Users\Rishu Yadav\AppData\Local\Temp\ipykernel_11044\1141008655.py:10: SyntaxWarning: invalid escape sequence '\P'
  test = pd.read_csv("E:\PROJECTS\ML PROJECTS\TITANIC DATASET EDA AND FEATURE ENGINEERING\test.csv")
C:\Users\Rishu Yadav\AppData\Local\Temp\ipykernel_11044\1141008655.py:13: SyntaxWarning: invalid escape sequence '\s'
  df['Title'] = df['Name'].str.extract(',\s*([^\.]+)\.', expand=False)
C:\Users\Rishu Yadav\AppData\Local\Temp\ipykernel_11044\11410086

OSError: [Errno 22] Invalid argument: 'E:\\PROJECTS\\ML PROJECTS\\TITANIC DATASET EDA AND FEATURE ENGINEERING\train (2).csv'